<a href="https://colab.research.google.com/github/KittLao/redshift_cluster/blob/main/redshift_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
import pandas as pd
import boto3
import json

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_to_cwd = "drive/MyDrive/Colab Notebooks/AWS/"

# Setup Configurations

Make sure dwh.cfg is populated with correct configurations and exists in same directory as this notebook.

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open(path_to_cwd + 'dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

# Setup needed Clients and Resource

In [ ]:
ec2 = boto3.resource(
    'ec2',
    region_name='us-east-1',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
  )

s3 = boto3.resource(
    's3',
    region_name='us-east-1',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
  )

iam = boto3.client(
    'iam',
    region_name='us-east-1',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
  )

redshift = boto3.client(
    'redshift',
    region_name='us-east-1',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
  )

In [ ]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
  print(obj)

# Setup IAM Role
This IAM role allows Redshift to be able to access S3 bucket ReadOnly.

Name of role is dwhRole, arn is arn:aws:iam::049086955797:role/dwhRole, and it has the policy AmazonS3ReadOnlyAccess.

In [ ]:
try:
  print("Creating IAM Role")
  # Came directly from AWS create IAM wizard.
  policy_statement = json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
  iam.create_role(Path="/",
                  RoleName=DWH_IAM_ROLE_NAME,
                  Description="IAM Role for Redshift Cluster accessing S3.",
                  AssumeRolePolicyDocument=policy_statement)
except Exception as e:
    print(e)

print("Attatching policy")
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess",
                        )['ResponseMetadata']['HTTPStatusCode']

print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)


# Create Redshift Cluster



In [ ]:
try:
  response = redshift.create_cluster(
      ClusterType=DWH_CLUSTER_TYPE,
      NodeType=DWH_NODE_TYPE,
      NumberOfNodes = int(DWH_NUM_NODES),
      # Identifiers and credentials
      DBName=DWH_DB,
      ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
      MasterUsername=DWH_DB_USER,
      MasterUserPassword=DWH_DB_PASSWORD,
      # Roles for s3 access
      IamRoles=[roleArn]
  )
except Exception as e:
  print(e)



In [ ]:
response

## Monitor Redshift cluster status

Wait untill ClusterStatus is Available

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

## Cluster endpoint and role ARN

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

# Open TCP port to access endpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

In [ ]:
!pip install psycopg2-binary

In [ ]:
DWH_ENDPOINT, DWH_PORT, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD

# Verify Connection to Cluster

In [ ]:
import psycopg2

try:
  print("Connecting to cluster")
  cnx = psycopg2.connect(
      host=DWH_ENDPOINT,
      port=DWH_PORT,
      database=DWH_DB,
      user=DWH_DB_USER,
      password=DWH_DB_PASSWORD)
  print("Connected to cluster")
except Exception as e:
  print(e)

In [ ]:
cursor = cnx.cursor()
cursor.execute('SELECT version()')
cursor.close()
cnx.close()

# Clean up
Remove cluster and detatch IAM Role policy.

In [ ]:
# Deletes cluster and saves snapshot
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)


In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# Detatch resources made for cluster such as IAM Role.
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)